<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/shaula0/Observable-Celestial-Object-List-Lookup/blob/92f3b6493dc0b9aa1e85dfe9138b59ba6569c97a/%EA%B4%80%EC%B8%A1%20%EA%B0%80%EB%8A%A5%20%EB%8C%80%EC%83%81%20%EC%A1%B0%ED%9A%8C.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

* 작성자: 정재영 (Jeong Jaeyoung)
* 이메일: shaula0@naver.com
* 작성일: 2025-03-10
* 설명: 천체 관측 가능한 대상을 계산하는 코드
* 요약: 일몰 시간 + 설정 시간(분) 부터 관측을 시작하며, 관측 종료 시간 까지 1시간 단위로 천체 목록을 조회하여, 설정 고도보다 높은 위치에 있을 경우 관측 가능 대상에 저장한다.

# pip install

In [ ]:
# !pip install astropy
# !pip insatll astroplan
# !pip install tabulate
# !pip install tqdm
# !pip install pandas
!pip install astroplan

In [ ]:
from astropy.coordinates import EarthLocation, AltAz, get_body, SkyCoord
from astropy.time import Time
from astroplan import Observer
from tabulate import tabulate
from tqdm import tqdm
import pandas as pd
import datetime
import numpy as np

# 변수 설정

In [ ]:
# 조회 날짜
start_date = datetime.date(2025, 1, 1)
end_date = datetime.date(2025, 12, 31)

# 관측 종료 시간(시, 분)
end_hour = 20
end_min = 30

# (태양계) 일몰 n분 후 부터 조회
planet_after_sunset_min = 30

# (태양계외 천체) 일몰 n분 후 부터 조회
deepsky_after_sunset_min = 70

# 관측 위치
loc = EarthLocation(lat=33.444558, lon=126.549283, height=394)  # 별빛누리 경도, 위도

# 관측 대상 고도 n이상
target_altitude = 25

# 조회 천체 목록 (태양계)
solar_objects = [
    "moon",     # 달
    "venus",    # 금성
    "mars",     # 화성
    "jupiter",  # 목성
    "saturn"    # 토성
]

# 조회 천체 목록 (태양계외 천체) (적경 RA, 적위 Dec)
non_solar_objects = [
    ["M31", (10.6847, 41.2692)],    # 안드로메다 은하
    ["M42", (83.8221, -5.3911)],    # 오리온 성운
    ["M45", (56.75, 24.1167)],      # 플레이아데스 성단
    ["M13", (250.423, 36.4613)],    # 헤라클레스 구상성단
    ["M57", (283.396, 33.0257)]     # 고리성운
]

## 메시에 목록 (숨김)

```
non_solar_objects = [
    ["M1", (83.6331, 22.0145)],    # 게 성운
    ["M2", (323.3624, -0.8233)],   # 구상성단
    ["M3", (205.5482, 28.3776)],   # 구상성단
    ["M4", (245.8964, -26.5256)],  # 구상성단
    ["M5", (229.6384, 2.0819)],    # 구상성단
    ["M6", (265.075, -32.2167)],   # 나비 성단
    ["M7", (267.92, -34.7833)],    # 프톨레마이오스 성단
    ["M8", (270.925, -24.38)],     # 라군 성운
    ["M9", (259.7992, -18.5164)],  # 구상성단
    ["M10", (254.2875, -4.1)],     # 구상성단
    ["M11", (282.7692, -6.2681)],  # 야생오리 성단
    ["M12", (251.8097, -1.9486)],  # 구상성단
    ["M13", (250.423, 36.4613)],   # 헤라클레스 구상성단
    ["M14", (264.4017, -3.2458)],  # 구상성단
    ["M15", (322.4933, 12.167)],   # 구상성단
    ["M16", (274.7, -13.8)],       # 독수리 성운
    ["M17", (275.325, -16.1717)],  # 오메가 성운
    ["M18", (275.625, -17.1333)],  # 산개성단
    ["M19", (255.6571, -26.268)],  # 구상성단
    ["M20", (270.66, -22.9722)],   # 삼렬 성운
    ["M21", (270.65, -22.5)],      # 산개성단
    ["M22", (280.46, -23.9)],      # 구상성단
    ["M23", (269.15, -19.0167)],   # 산개성단
    ["M24", (274.25, -18.5)],      # 성운형 성단
    ["M25", (277.96, -19.1167)],   # 산개성단
    ["M26", (281.05, -9.3833)],    # 산개성단
    ["M27", (299.898, 22.7216)],   # 아령 성운
    ["M28", (276.1362, -24.8695)], # 구상성단
    ["M29", (305.8833, 38.525)],   # 산개성단
    ["M30", (325.0929, -23.1792)], # 구상성단
    ["M31", (10.6847, 41.2692)],   # 안드로메다 은하
    ["M32", (10.6743, 40.8667)],   # 안드로메다 위성 은하
    ["M33", (23.4621, 30.66)],     # 삼각형자리 은하
    ["M34", (40.5167, 42.7833)],   # 산개성단
    ["M35", (92.375, 24.3333)],    # 산개성단
    ["M36", (84.05, 34.1333)],     # 산개성단
    ["M37", (88.0833, 32.55)],     # 산개성단
    ["M38", (81.117, 35.85)],      # 산개성단
    ["M39", (323.4, 48.4333)],     # 산개성단
    ["M40", (184.7883, 58.0833)],  # 쌍성
    ["M41", (101.5, -20.75)],      # 산개성단
    ["M42", (83.8221, -5.3911)],   # 오리온 성운
    ["M43", (83.8, -5.2833)],      # 오리온 성운 일부
    ["M44", (130.0667, 19.6667)],  # 프레세페 성단
    ["M45", (56.75, 24.1167)],     # 플레이아데스 성단
    ["M46", (115.45, -14.8167)],   # 산개성단
    ["M47", (115.3, -14.4833)],    # 산개성단
    ["M48", (123.55, -5.75)],      # 산개성단
    ["M49", (187.4462, 8.0004)],   # 처녀자리 타원은하
    ["M50", (104.05, -8.3333)],    # 산개성단
    ["M51", (202.4696, 47.1952)],  # 부자 은하 (소용돌이 은하)
    ["M52", (351.0537, 61.5919)],  # 산개성단
    ["M53", (198.2308, 18.1697)],  # 구상성단
    ["M54", (283.764, -30.4794)],  # 구상성단
    ["M55", (294.9979, -30.964)],  # 구상성단
    ["M56", (289.1474, 30.1842)],  # 구상성단
    ["M57", (283.396, 33.0257)],   # 고리 성운
    ["M58", (189.4246, 11.8183)],  # 나선 은하
    ["M59", (190.0075, 11.6464)],  # 타원 은하
    ["M60", (190.9167, 11.5528)],  # 타원 은하
    ["M61", (185.4783, 4.4736)],   # 나선 은하
    ["M62", (255.3038, -30.1111)], # 구상성단
    ["M63", (198.9517, 42.0292)],  # 해바라기 은하
    ["M64", (194.1825, 21.6839)],  # 검은눈 은하
    ["M65", (169.6656, 13.0924)],  # 나선 은하
    ["M66", (170.0625, 12.9914)],  # 나선 은하
    ["M67", (132.85, 11.8)],       # 산개성단
    ["M68", (189.8667, -26.7447)], # 구상성단
    ["M69", (277.8467, -32.3472)], # 구상성단
    ["M70", (279.1, -32.3)],       # 구상성단
    ["M71", (298.442, 18.78)],     # 구상성단
    ["M72", (313.3667, -12.5333)], # 구상성단
    ["M73", (314.6, -12.6333)],    # 성단?
    ["M74", (24.1725, 15.7836)],   # 나선 은하
    ["M75", (300.0833, -21.9167)], # 구상성단
    ["M76", (25.5354, 51.5742)],   # 작은 아령 성운
    ["M77", (40.6696, -0.0133)],   # 나선 은하
    ["M78", (83.0833, 0.05)],      # 반사 성운
    ["M79", (81.0421, -24.5244)],  # 구상성단
    ["M80", (244.26, -22.976)],    # 구상성단
    ["M81", (148.8882, 69.0653)],  # 보데 은하
    ["M82", (148.9697, 69.6797)],  # 시가 은하
    ["M83", (204.2533, -29.865)],  # 남쪽 바람개비 은하
    ["M84", (190.9146, 12.8869)],  # 렌즈형 은하
    ["M85", (186.3500, 18.1914)],  # 타원 은하
    ["M86", (190.9150, 12.9452)],  # 렌즈형 은하
    ["M87", (187.7059, 12.3911)],  # 거대 타원 은하 (블랙홀 포함)
    ["M88", (190.1950, 14.4204)],  # 나선 은하
    ["M89", (190.3921, 12.5569)],  # 타원 은하
    ["M90", (190.7521, 13.1625)],  # 나선 은하
    ["M91", (188.8283, 14.4969)],  # 막대나선 은하
    ["M92", (259.2812, 43.1367)],  # 구상성단
    ["M93", (116.1525, -23.8569)], # 산개성단
    ["M94", (192.7212, 41.1197)],  # 나선 은하
    ["M95", (161.8875, 11.7033)],  # 막대나선 은하
    ["M96", (161.9625, 11.8208)],  # 나선 은하
    ["M97", (168.6986, 55.0194)],  # 올빼미 성운
    ["M98", (181.3667, 14.9000)],  # 나선 은하
    ["M99", (185.0250, 14.4167)],  # 나선 은하
    ["M100", (186.5354, 15.8211)], # 나선 은하
    ["M101", (210.8021, 54.3489)], # 부자 은하
    ["M102", (226.6250, 55.7633)], # 렌즈형 은하
    ["M103", (23.3925, 60.7392)],  # 산개성단
    ["M104", (189.9971, -11.6231)],# 소머리 은하
    ["M105", (161.9562, 12.5819)], # 타원 은하
    ["M106", (184.7396, 47.3036)], # 나선 은하
    ["M107", (248.1333, -13.0517)],# 구상성단
    ["M108", (167.7983, 55.6733)], # 나선 은하
    ["M109", (179.3954, 53.3747)], # 나선 은하
    ["M110", (10.0853, 41.6853)]   # 안드로메다 위성 은하
]
```

# 관측 가능 천체 검색

In [ ]:
# 관측자 설정 (UTC 기준)
observer = Observer(location=loc)

# 데이터프레임 초기화
df = pd.DataFrame(columns=["날짜", "일몰시간", "관측가능 대상"])

def deg_to_dms(degrees):
    """십진수 도(deg)를 도(D) 분(M) 초(S) 형태로 변환"""
    d = int(degrees)
    m = int((degrees - d) * 60)
    s = (degrees - d - m / 60) * 3600
    return f"{d}° {m}' {s:.2f}\""

# 설정한 날짜 범위 for문
for single_date in tqdm(range((end_date - start_date).days + 1)):
    #검색 날짜
    current_date = start_date + datetime.timedelta(days=single_date)

    # 해당 날짜의 일몰 시간 계산 (UTC 기준)
    sunset_time_utc = observer.sun_set_time(Time(current_date.strftime("%Y-%m-%d")), which="nearest")

    # UTC → KST 변환 (UTC + 9시간)
    sunset_time_kst = sunset_time_utc + datetime.timedelta(hours=9)

    # 관측 종료 시간 KST → UTC 변환 (KST - 9시간)
    end_time_utc = Time(datetime.datetime.combine(current_date, datetime.time(end_hour, end_min)) + datetime.timedelta(hours=-9))

    # 조건에 맞는 관측 대상 담을 list 초기화
    observable_list = []

    # 태양계 반복문
    for name in solar_objects:
        # 관측 시작 시간 = 일몰 시간 + 일몰 후 설정 시간
        current_time_utc = sunset_time_utc + datetime.timedelta(minutes = planet_after_sunset_min)

        # 관측 끝 시간 - 관측 시작 시간
        difference_time = end_time_utc - current_time_utc

        # 분 단위로 변환
        difference_time_to_min = difference_time.to_value('min')

        # 관측 종료 시간이 일몰시간보다 빠른지 확인
        if difference_time_to_min >= 0:
            # 관측 시작 시간 + 1h > 관측 끝 시간 만큼 반복문
            for _ in range(int(difference_time_to_min//60) + 2):
                # 관측 마지막 시간 보다 크다면 마지막 설정 시간으로 계산
                if current_time_utc >= end_time_utc:
                    current_time_utc = end_time_utc

                # 좌표계 생성
                altaz = AltAz(location=loc, obstime=current_time_utc)

                # 고도 계산
                altitude = get_body(name, current_time_utc, loc).transform_to(altaz).alt.deg

                # 설정 고도 이상일 경우 행성 이름 추가 후 종료
                if altitude >= target_altitude:
                    observable_list.append(name)
                    break

                # 시간 +1h
                current_time_utc += datetime.timedelta(hours=1)
        else:
            print(current_time_utc.strftime("%Y-%m-%d") + "은 관측 종료 시간보다 일몰시간이 빠릅니다.")

    # 태양계외 천체 반복문
    for name, (ra, dec) in non_solar_objects:
        # Messier Object의 적경(RA), 적위(Dec) 설정
        obj_coord = SkyCoord(ra=ra, dec=dec, unit="deg")

        # 관측 시작 시간 = 일몰 시간 + 일몰 후 설정 시간
        current_time_utc = sunset_time_utc + datetime.timedelta(minutes = deepsky_after_sunset_min)

        # 관측 끝 시간 - 관측 시작 시간
        difference_time = end_time_utc - current_time_utc

        # 분 단위로 변환
        difference_time_to_min = difference_time.to_value('min')

        # 관측 종료 시간이 일몰시간보다 빠른지 확인
        if difference_time_to_min >= 0:
            # 관측 시작 시간 + 1h > 관측 끝 시간 만큼 반복문
            for _ in range(int(difference_time_to_min // 60) + 2):
                # 관측 마지막 시간보다 크다면 마지막 설정 시간으로 계산
                if current_time_utc >= end_time_utc:
                    current_time_utc = end_time_utc

                # 좌표계 생성
                altaz = AltAz(location=loc, obstime=current_time_utc)

                # 고도 계산
                altitude = obj_coord.transform_to(altaz).alt.deg

                # 설정 고도 이상일 경우 행성 이름 추가 후 종료
                if altitude >= target_altitude:
                    observable_list.append(name)
                    break

                # 시간 +1h
                current_time_utc += datetime.timedelta(hours=1)
        else:
            print(current_time_utc.strftime("%Y-%m-%d") + "은 관측 종료 시간보다 일몰시간이 빠릅니다.")

    # 현재 날짜에 관측 가능한 대상 리스트를 DataFrame 형태로 변환
    df_observable_list = pd.DataFrame([[current_date.strftime("%Y-%m-%d"), sunset_time_kst.strftime("%H:%M"), observable_list]], columns=["날짜", "일몰시간", "관측가능 대상"])

    # df에 추가
    df = pd.concat([df, df_observable_list])

# print(tabulate(df, headers='keys', tablefmt='psql', showindex=True))

In [ ]:
print(tabulate(df, headers='keys', tablefmt='psql', showindex=True))

In [ ]:
# excel 저장
df.to_excel(start_date.strftime("%y%m%d") + "-" + end_date.strftime("%y%m%d") + "_Sunset+30m-" + str(end_hour).zfill(2) + "h" + str(end_min).zfill(2) + "m_Alt" + str(target_altitude) + "deg+" + ".xlsx", index=False)